In [ ]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import mediapipe as mp
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder


def extract_rgb_frames(video_folder, output_base_folder, frame_step=1, target_size=(320, 240)):
    
    video_files = [f for f in os.listdir(video_folder) if f.endswith(('.avi', '.mp4'))]
    for video_file in video_files:
        video_path = os.path.join(video_folder, video_file)
        output_folder = os.path.join(output_base_folder, os.path.splitext(video_file)[0])
        os.makedirs(output_folder, exist_ok=True)
        cap = cv.VideoCapture(video_path)
        frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
        
        for i in range(0, frame_count, frame_step):
            cap.set(cv.CAP_PROP_POS_FRAMES, i)
            ret, frame = cap.read()
            if not ret:
                break
            h, w, _ = frame.shape
            rgb_frame = frame[:, w//2:, :] if w >= 2*h else frame
            frame_resized = cv.resize(rgb_frame, target_size, interpolation=cv.INTER_AREA)
            cv.imwrite(os.path.join(output_folder, f"frame_{i:04d}.jpg"), frame_resized)
        cap.release()


extract_rgb_frames("video/fall", "frames_fall")
extract_rgb_frames("video/adl", "frames_adl")

In [ ]:
SELECTED_LANDMARKS = [0, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
mpPose = mp.solutions.pose
pose = mpPose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8)
acceleration_queue = deque(maxlen=5)

def detectPose(image, prev_landmarks, prev_velocity, time_interval):
    """Detect human pose landmarks and calculate velocity/acceleration."""
    imgHeight, imgWidth, _ = image.shape
    results = pose.process(cv.cvtColor(image, cv.COLOR_BGR2RGB))
    landmarks, neck_y = [], None
    
    if results.pose_landmarks:
        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            if idx in SELECTED_LANDMARKS:
                landmarks.append((landmark.x * imgWidth, landmark.y * imgHeight))
        left_shoulder, right_shoulder = results.pose_landmarks.landmark[11], results.pose_landmarks.landmark[12]
        neck_y = (left_shoulder.y + right_shoulder.y) / 2 * imgHeight
    
    current_velocity = np.array(landmarks) - np.array(prev_landmarks) if prev_landmarks else None
    acceleration = (current_velocity - prev_velocity) / time_interval if prev_velocity is not None else None
    if acceleration is not None:
        acceleration_queue.append(np.mean(np.abs(acceleration)))
    
    return results, landmarks, neck_y, current_velocity, acceleration

def classify_pose(neck_y, image_height):
    
    return "FALL" if neck_y and neck_y > 0.6 * image_height and np.mean(acceleration_queue) > 5 else "ADL"

def processFrames(input_folders, output_csv):
   
    data = []
    for label, folder_path in input_folders.items():
        for subfolder in os.listdir(folder_path):
            prev_landmarks, prev_velocity = None, None
            for filename in sorted(os.listdir(os.path.join(folder_path, subfolder))):
                image = cv.imread(os.path.join(folder_path, subfolder, filename))
                if image is None:
                    continue
                _, landmarks, neck_y, current_velocity, acceleration = detectPose(image, prev_landmarks, prev_velocity, 1/30)
                if landmarks:
                    data.append([coord for lm in landmarks for coord in lm] + [classify_pose(neck_y, image.shape[0])])
                prev_landmarks, prev_velocity = landmarks, current_velocity
    
    columns = [f'{axis}{i+1}' for i in range(len(SELECTED_LANDMARKS)) for axis in ['x', 'y']] + ['Label']
    pd.DataFrame(data, columns=columns).to_csv(output_csv, index=False)

processFrames({'adl': 'frames_adl', 'fall': 'frames_fall'}, 'pose_data_2d.csv')

In [ ]:
train_df = pd.read_csv('pose_data_2d.csv')
features, labels = train_df.iloc[:, :-1].values, train_df['Label'].values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

def create_sequences(features, labels, seq_length, stride=6):
    """Create time-series sequences for LSTM training."""
    X, y = [], []
    for i in range(0, len(features) - seq_length + 1, stride):
        X.append(features[i:i + seq_length])
        y.append(1 if sum(labels[i:i + seq_length]) > (seq_length // 2) else 0)
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(features, labels, 12)

model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(12, X_train.shape[2])),
    Dropout(0.2),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    TimeDistributed(Dense(32, activation='relu')),
    TimeDistributed(Dense(1, activation='sigmoid'))
])
model.compile(optimizer=Adam(0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.2, batch_size=16, epochs=100, verbose=1)
model.save('lstm_fall_detection.h5')


In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('pose_data_2d.csv')

In [5]:
df

,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,y9,x10,y10,x11,y11,x12,y12,x13,y13,Label
0,218.840389,71.376386,232.011280,83.661053,208.338223,86.556129,235.739212,103.860483,203.824768,105.510864,...,123.351860,226.700211,153.019495,215.970249,153.865285,235.494976,181.419783,226.246281,179.755883,ADL
1,218.885441,71.438642,231.514854,83.658800,207.841396,86.555994,235.150108,102.952058,203.835716,105.283706,...,122.913923,228.122959,147.433691,216.703777,147.923183,233.504276,171.142416,224.974098,171.344275,ADL
2,218.877220,71.798480,230.577774,84.169228,207.366257,86.581850,234.275055,102.901053,203.619556,104.991510,...,122.363405,228.286896,146.965857,217.161121,146.931024,232.316761,169.727697,224.460030,170.392728,ADL
3,218.504314,71.835101,229.681282,84.745152,206.811562,86.555600,233.752575,102.921088,202.848949,104.115150,...,121.326013,227.649193,145.537477,216.763611,146.178389,227.152843,168.135653,220.189342,169.720430,ADL
4,218.217678,71.859870,229.365692,85.082502,206.720066,86.529858,233.381004,102.907977,202.256336,102.820566,...,121.181674,227.309856,144.787374,216.483860,145.787630,223.253593,167.065701,218.380909,167.549200,ADL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6591,139.888887,154.730759,146.070890,167.319274,129.310532,153.760028,146.507244,181.278605,118.344727,146.172738,...,169.824543,142.692366,151.709762,130.344315,140.924406,128.118229,141.828518,119.724445,138.894296,FALL
6592,136.814070,154.827404,145.518541,168.331532,129.752169,153.363619,145.451450,183.314180,117.870550,146.636810,...,169.447360,140.149717,150.743036,132.126360,140.326052,127.627449,144.558249,120.369644,140.311375,FALL
6593,138.252106,153.769069,144.197187,167.235932,129.915495,154.777393,143.253994,184.517670,117.972965,149.385395,...,169.827676,137.533817,143.862176,131.811886,138.700190,123.090754,141.297741,119.743338,140.938854,FALL
6594,133.293200,153.915081,140.390778,166.996536,130.293484,154.753890,142.628841,184.390182,118.889055,150.451956,...,166.576824,136.590443,145.529523,133.049135,139.199238,123.439655,142.310557,122.079067,142.426229,FALL
